# Representation

In [1]:
import pandas as pd
import numpy as np

### OCR Factor $\psi_o$

These factors capture the predictions of a character-based OCR system, and hence exist between every image variable and its corresponding character variable. The number of these factors of word w is $len(w)$. The value of factor between an image variable and the character variable at position i is dependent on $img(i)$ and $char(i)$

$$\psi_o (img(i)=id,char(i)=a) = prob$$

In [2]:
ocr_table_factor = pd.read_table("dataset/ocr.dat", names=["id", "character", "p"], index_col=["id", "character"])
ocr_table_factor

p
id  character          
0   d          0.153411
    o          0.088148
    i          0.011325
    r          0.074593
    a          0.096568
    h          0.130159
    t          0.131382
    n          0.012166
    s          0.073045
    e          0.229204
1   d          0.096485
    o          0.006723
    i          0.230524
    r          0.082828
    a          0.055986
    h          0.003760
    t          0.265633
    n          0.070771
    s          0.031128
    e          0.156162
2   d          0.199828
    o          0.038723
    i          0.015480
    r          0.014704
    a          0.202446
    h          0.084658
    t          0.023568
    n          0.140544
    s          0.187118
    e          0.092932
...                 ...
997 d          0.009863
    o          0.153503
    i          0.017599
    r          0.133825
    a          0.004070
    h          0.201336
    t          0.044256
    n          0.096157
    s          0.193126
    e          0.146267
998 d          0.022313
    o          0.140012
    i          0.005391
    r          0.216242
    a          0.081150
    h          0.263588
    t          0.050053
    n          0.021860
    s          0.093046
    e          0.106346
999 d          0.153038
    o          0.082118
    i          0.140054
    r          0.003380
    a          0.112325
    h          0.059706
    t          0.053938
    n          0.058457
    s          0.141211
    e          0.195774

[10000 rows x 1 columns]

In [3]:
def flat(X):
    return [item for sublist in X for item in sublist]   

def ocr_factor(X):
    """ocr factor
    
    :type X: list of list
    :param list X: list of assignments for each sentence
    :return the probability of factor
    """
    X = flat(X)
    return np.exp(np.sum(np.log([ocr_table_factor.ix[item["img"], item["char"]]["p"] for item in X])))
print ocr_factor([[{"img": 1, "char": "e"}, {"img": 2, "char": "d"}]])
print ocr_factor([[{"img": 1, "char": "e"}, {"img": 2, "char": "d"}]])

0.031205540136
0.031205540136


### Transition Factors $\psi_t$

Since we also want to represent the co-occurence frequencies of the characters in our model, we add these factors between all consecutive character variables. The number of these factors of word $w$ is $len(w)-1$. The value of factor between two character variables at positions $i$ and $i+1$ is dependent on $char(i)$ and $char(i+1)$, and is high if $char(i+1)$ is frequently preceded by $char(i)$ in english words

$$\psi_t(char(i)=a, char(i+1)=b) = prob$$

In [4]:
transition_table_factor = pd.read_table("dataset/trans.dat", names=["current", "next", "p"], index_col=["current", "next"])
transition_table_factor

p
current next          
n       s     0.356108
t       r     0.405734
        s     0.286997
s       o     0.391206
i       e     0.386189
s       r     0.187065
r       a     0.458026
n       d     0.401417
e       t     0.425656
o       n     0.452152
h       n     0.187065
t       n     0.214882
o       h     0.252718
t       o     0.431759
i       a     0.364271
e       i     0.340564
n       i     0.379802
e       s     0.422828
a       e     0.314818
        n     0.474042
h       a     0.408239
n       h     0.233033
d       a     0.404029
        s     0.224689
e       d     0.436908
o       t     0.411486
r       d     0.331590
d       e     0.474945
h       e     0.444204
r       i     0.437535
...                ...
a       i     0.401417
        d     0.424252
o       a     0.356108
e       a     0.427729
        h     0.276066
o       r     0.449938
        d     0.383048
a       o     0.279902
d       r     0.314818
o       s     0.402295
t       a     0.465580
s       e     0.474494
r       o     0.437535
h       o     0.417688
s       i     0.419922
r       n     0.354672
s       h     0.440617
h       d     0.187065
s       t     0.477175
r       t     0.385153
o       o     0.388227
e       r     0.532836
t       e     0.512932
e       o     0.276066
s       n     0.331590
d       o     0.406576
s       a     0.410683
        s     0.375292
d       t     0.187065
o       i     0.343904

[100 rows x 1 columns]

In [5]:
def trans_factor(X):
    """trans factor
    
    :type X: list of list
    :param list X: list of assignments for each sentence
    
    :return the probability of factor
    """
    p = [trans_factor_sentence(s) for s in X]
    return np.exp(np.sum(np.log(p)))
        
def trans_factor_sentence(X):
    """trans factor
    
    :type X: list of list
    :param list X: list of assignments for each sentence
    
    :return the probability of factor
    """
    return np.exp(np.sum(np.log([transition_table_factor.ix[X[i]["char"], X[i+1]["char"]]["p"] for i in range(0, len(X) - 1)])))

print trans_factor_sentence([{"img": 1, "char": "e"}, {"img": 2, "char": "d"}, {"img": 1, "char": "e"}, {"img": 2, "char": "d"}])
print trans_factor([[{"img": 1, "char": "e"}, {"img": 2, "char": "d"}], [{"img": 1, "char": "e"}, {"img": 2, "char": "d"}]])

0.0906615863474
0.190888600464


### Skip Factors $\psi_s$

We would like to capture in our model is that similar images in a word always represent the same character. Thus our model score should be higher if it predicts the same characters for similar images. These factors exist between every pair of image variables that have the same id, i.e. this factor exist between all $i$,$j$, $i \ne j$ such that $img(i)==img(j)$. The value of this factor depends on $char(i)$ and $char(j)$, and is $5.0$ if $char(i)==char(j)$, and $1.0$ otherwise.

In [6]:
import itertools
def skip_factor(X):
    """trans factor
    
    :type X: list of dictionary
    :param list X: list of assignment 
    
    :return the probability of factor
    """
    X = flat(X)
    result = 1
    pairs = itertools.combinations(X, 2)
    for i, j in pairs:
        if i['img'] == j['img'] and i["char"] == j["char"]:
            result = result * 5
    return result
print skip_factor([[{"img": 1, "char": "e"}, {"img": 1, "char": "d"}]]) 
print skip_factor([[{"img": 1, "char": "e"}, {"img": 1, "char": "e"}]]) 
print skip_factor([[{"img": 1, "char": "e"}, {"img": 1, "char": "e"}], [{"img": 2, "char": "d"}, {"img": 2, "char": "e"}]]) 
print skip_factor([[{"img": 1, "char": "e"}, {"img": 1, "char": "e"}], [{"img": 2, "char": "d"}, {"img": 2, "char": "d"}]]) 

1
5
5
25


In [7]:
class Model:
    def __init__(self, factors=None):
        self.factors = factors
        
    def score(self, X):
        p = [factor(X) for factor in self.factors]
        return np.exp(np.sum(np.log(p)))
    
    @staticmethod
    def make_assignment(list_char, word_image):
        return [[{"char": a[0], "img": a[1]} for a in zip(list_char, word_image)]]
    
    def predict(self, word_image):
        chars = "etaoinshrd"
        assignments = list(itertools.product(chars, repeat=len(word_image)))
        assignments = [self.make_assignment(list_char, word_image) for list_char in assignments]
        scores = [(assignment, self.score(assignment)) for assignment in assignments]
        best = max(scores, key=lambda score: score[1])
        return best

In [8]:
model_1 = Model(factors=[ocr_factor])
model_2 = Model(factors=[ocr_factor, trans_factor])
model_3 = Model(factors=[ocr_factor, trans_factor, skip_factor])

In [9]:
print model_1.score([[{'char': 'd', 'img': 582}, {'char': 'a', 'img': 969}, {'char': 'd', 'img': 582}]])
print model_1.score([[{'char': 'd', 'img': 582}, {'char': 'd', 'img': 969}, {'char': 'd', 'img': 582}]])
print model_1.score([[{'char': 'a', 'img': 582}, {'char': 'a', 'img': 969}, {'char': 'd', 'img': 582}]])
print model_1.score([[{'char': 'a', 'img': 582}, {'char': 'a', 'img': 969}, {'char': 'a', 'img': 582}]])

0.00534435069649
0.0050133738879
0.00528119593478
0.00521878747962


In [10]:
X1 = [[{"img": 582, "char": "e"}, {"img": 969, "char": "d"}], [{"img": 582, "char": "a"}, {"img": 969, "char": "d"}]]
X2 = [[{"img": 582, "char": "a"}, {"img": 969, "char": "d"}], [{"img": 582, "char": "a"}, {"img": 969, "char": "e"}]]
X3 = [[{"img": 582, "char": "a"}, {"img": 969, "char": "d"}], [{"img": 582, "char": "a"}, {"img": 969, "char": "d"}]]

In [11]:
print model_1.score(X1)
print model_2.score(X1)
print model_3.score(X1)

0.000181006539499
3.35512079554e-05
0.000167756039777


In [12]:
print model_1.score(X2)
print model_2.score(X2)
print model_3.score(X2)

0.000701101885302
9.36406864829e-05
0.000468203432415


In [13]:
print model_1.score(X3)
print model_2.score(X3)
print model_3.score(X3)

0.000696681170396
0.00012539547631
0.00313488690776


# Inference

Using the graphical model, write code to perform exhaustive inference, i.e. your code should be able to calculate the probability of any assignment of the character and image variables. To calculate the normalization constant Z for the word w, you will need to go through all possible assignments to the character variables (there will be $10^{len(w)}$ of these).

In [14]:
X = [[int(t) for t in s.split()] for s in open("dataset/data.dat", "r").readlines()]
X[:5]

[[582, 969, 582, 969],
 [322, 959, 959, 880, 555, 959],
 [92, 809, 794, 92, 231, 990],
 [942, 979, 793],
 [942, 567, 942]]

In [15]:
word_image = X[0]
%timeit -r 1 model_1.predict(word_image)
%timeit -r 1 model_2.predict(word_image)
%timeit -r 1 model_3.predict(word_image)

1 loop, best of 1: 6.59 s per loop
1 loop, best of 1: 11.4 s per loop
1 loop, best of 1: 11.8 s per loop


In [16]:
word_image = X[1]
%timeit -r 1 model_1.predict(word_image)
%timeit -r 1 model_2.predict(word_image)
%timeit -r 1 model_3.predict(word_image)

1 loop, best of 1: 16min 45s per loop
1 loop, best of 1: 30min 3s per loop
1 loop, best of 1: 29min 33s per loop


In [20]:
word_image = X[3]
print model_1.predict(word_image)
print model_2.predict(word_image)
print model_3.predict(word_image)

([[{'char': 'a', 'img': 942}, {'char': 'd', 'img': 979}, {'char': 's', 'img': 793}]], 0.014222639030861513)
([[{'char': 'a', 'img': 942}, {'char': 'd', 'img': 979}, {'char': 'o', 'img': 793}]], 0.0023411548440003252)
([[{'char': 'a', 'img': 942}, {'char': 'd', 'img': 979}, {'char': 'o', 'img': 793}]], 0.0023411548440003252)


In [5]:
import time
%timeit time.sleep(5)

1 loop, best of 3: 5 s per loop
